In [1]:
import pandas as pd
from retrying import retry
import numpy as np
from geopy.geocoders import Nominatim
import os
import time
import pickle
import sqlite3

In [2]:
class Cache(object):
    def __init__(self, fn='cache.db'):
       self.conn = conn = sqlite3.connect(fn)
       cur = conn.cursor()
       cur.execute('CREATE TABLE IF NOT EXISTS '
                   'Geo ( '
                   'address STRING PRIMARY KEY, '
                   'latitude REAL NOT NULL,'
                   'longitude REAL NOT NULL'
                   ')')
       conn.commit()

    def address_cached(self, lat,long):
        cur = self.conn.cursor()
        cur.execute('SELECT address FROM Geo WHERE latitude=? and longitude=?', (lat,long))
        res = cur.fetchone()
        if res is None: return False
        return pickle.loads(res[0])

    def save_to_cache(self, address, lat,long):
        cur = self.conn.cursor()
        cur.execute('INSERT INTO Geo(address, latitude,longitude) VALUES(?, ?, ?)',
                    (address, lat, long))
        self.conn.commit()

@retry(stop_max_attempt_number=15)
def getRoad(samplecord):
    cache = Cache()
    location = cache.address_cached(samplecord[0],samplecord[1])
    if location:
        return location.raw['address']['road']
    else:
        location = geolocator.reverse(samplecord)
        cache.save_to_cache(location.raw['address']['road'],samplecord[0],samplecord[1])
        time.sleep(1)
        return location.raw['address']['road']

geolocator = Nominatim()
Files=os.listdir('../Excel')
dir_path = os.path.dirname(os.path.realpath('.'))
for file in Files:
    fname=dir_path+"\Excel\\"+file
    df=pd.read_csv(fname)
    roads=[]
    count=0
    for sample in df['WKT']:
        processedarray=[]
        sample=sample.replace("LINESTRING (", "")
        sample=sample.replace(")" ,"")
        coordinates=sample.split(",")
        for coord in coordinates:
            tmp=coord.split()
            tmp=[t.strip() for t in tmp]
            tmp[0],tmp[1]=tmp[1],tmp[0]
            tmp=[float(t) for t in tmp]
            processedarray.append(tuple(tmp))
        samplecord=processedarray[int(len(processedarray)/2)]
        road=getRoad(samplecord)
        d={'FullName':road,'Co-ordinates(x,y)':str(samplecord[0])+","+str(samplecord[1])}
        roads.append(d)
        count+=1
        print(count,end="")
        print(d)
    df2=pd.DataFrame(roads)
    r=pd.concat([df, df2],axis=1)
    f=fname+'\.csv'
    r.to_csv(f)

GeocoderTimedOut: Service timed out